# **Google Imagen 4.0 Upscaler (Cloud API)**

AI-powered image upscaling using Google's Imagen 4.0 on Vertex AI.

## Advantages
- **No local GPU required** - runs in Google Cloud
- **AI-enhanced upscaling** - not just interpolation
- **Simple setup** - just need Google Cloud credentials

## Requirements
1. Google Cloud account with billing enabled
2. Vertex AI API enabled
3. `gcloud` CLI installed and authenticated

## Quick Setup
```bash
# Enable APIs
gcloud services enable aiplatform.googleapis.com

# Authenticate
gcloud auth application-default login
```

## Upscale Factors
- `x2` - Double resolution
- `x3` - Triple resolution  
- `x4` - Quadruple resolution

In [1]:
# @title Setup Environment (Run Once)
import subprocess
import sys
import os
from pathlib import Path

# Get repo root
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

# Directory paths
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"

# Create directories
for d in [OUTPUT_DIR, INPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Output directory: {OUTPUT_DIR}")

def install_packages():
    """Install required packages for Imagen API."""
    packages = [
        'google-auth',
        'google-cloud-aiplatform',
        'requests',
        'pillow',
    ]
    
    for package in packages:
        try:
            subprocess.run(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                check=True, capture_output=True
            )
            print(f"\u2713 {package} installed")
        except subprocess.CalledProcessError as e:
            print(f"\u2717 Error installing {package}")

print("Installing packages...")
install_packages()
print("\n\u2705 Environment Setup Complete!")

Repo root: C:\Users\Armaan\Desktop\Artinafti
Output directory: C:\Users\Armaan\Desktop\Artinafti\4xoutput
Installing packages...
✓ google-auth installed
✓ google-cloud-aiplatform installed
✓ requests installed
✓ pillow installed

✅ Environment Setup Complete!


In [2]:
# @title Authenticate with Google Cloud
import google.auth
import google.auth.transport.requests
from google.oauth2 import service_account
import os

def get_credentials():
    """
    Get Google Cloud credentials.
    
    Tries in order:
    1. Application Default Credentials (from gcloud auth application-default login)
    2. Service account key file (if SERVICE_ACCOUNT_KEY_PATH is set)
    """
    try:
        # Try Application Default Credentials first
        credentials, project = google.auth.default(
            scopes=['https://www.googleapis.com/auth/cloud-platform']
        )
        print(f"\u2713 Using Application Default Credentials")
        print(f"  Project: {project}")
        return credentials, project
    except Exception as e:
        print(f"\u2717 Could not get default credentials: {e}")
        print("\nPlease run: gcloud auth application-default login")
        raise

def get_access_token(credentials):
    """Get fresh access token from credentials."""
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    return credentials.token

# Get credentials
credentials, detected_project = get_credentials()
print("\n\u2705 Authentication successful!")

✓ Using Application Default Credentials
  Project: artinafti

✅ Authentication successful!


In [3]:
# @title Configuration
import os
from pathlib import Path

# ============== GOOGLE CLOUD SETTINGS ==============
# Your Google Cloud Project ID
PROJECT_ID = "artinafti"  # Replace if not auto-detected

# Region for Vertex AI (us-central1 recommended)
REGION = "us-central1"

# ============== BATCH INPUT WITH CUSTOM OUTPUT SIZES ==============
# Each image has its own target print size (width x height in inches)
DPI = 150

IMAGE_CONFIGS = [
    {
        "image_id": "7a49a140-beee-4141-9593-3fddb9194240",
        "input_path": "input/7a49a140-beee-4141-9593-3fddb9194240.jpg",
        "width_inches": 10,
        "height_inches": 20,
    },
    {
        "image_id": "b46bc519-9b4b-487f-b8d4-b95195d7e02e",
        "input_path": "input/b46bc519-9b4b-487f-b8d4-b95195d7e02e.jpg",
        "width_inches": 17,
        "height_inches": 10,
    },
    {
        "image_id": "df5cc4ff-9846-412c-ad72-e53c42184b9c",
        "input_path": "input/df5cc4ff-9846-412c-ad72-e53c42184b9c.jpg",
        "width_inches": 15,
        "height_inches": 12,
    },
]

# Calculate target pixel dimensions
for config in IMAGE_CONFIGS:
    config["target_width"] = config["width_inches"] * DPI
    config["target_height"] = config["height_inches"] * DPI

# ============== UPSCALE SETTINGS ==============
# Upscale factor: "x2", "x3", or "x4"
UPSCALE_FACTOR = "x4"

# Output format: "image/png" or "image/jpeg"
OUTPUT_MIME_TYPE = "image/png"

# JPEG compression quality (0-100, only used for JPEG output)
COMPRESSION_QUALITY = 90

# Prompt for upscaling (can influence output quality)
UPSCALE_PROMPT = "Upscale the image with high quality and sharp details"

# ============== PATH SETTINGS ==============
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

OUTPUT_DIR = REPO_ROOT / "4xoutput"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Configuration set!")
print(f"Project ID: {PROJECT_ID}")
print(f"Region: {REGION}")
print(f"Upscale factor: {UPSCALE_FACTOR}")
print(f"Output format: {OUTPUT_MIME_TYPE}")
print(f"DPI: {DPI}")
print(f"\nImages to process:")
for config in IMAGE_CONFIGS:
    print(f"  {config['image_id'][:20]}...: {config['width_inches']}x{config['height_inches']}\" -> {config['target_width']}x{config['target_height']}px")

Configuration set!
Project ID: artinafti
Region: us-central1
Upscale factor: x4
Output format: image/png
DPI: 150

Images to process:
  7a49a140-beee-4141-9...: 10x20" -> 1500x3000px
  b46bc519-9b4b-487f-b...: 17x10" -> 2550x1500px
  df5cc4ff-9846-412c-a...: 15x12" -> 2250x1800px


In [4]:
# @title Upscale Functions
import base64
import requests
import json
import time
from pathlib import Path
from PIL import Image
import io

def encode_image_to_base64(image_path: str) -> str:
    """Read image file and encode to base64."""
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def decode_base64_to_image(base64_string: str) -> Image.Image:
    """Decode base64 string to PIL Image."""
    image_data = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(image_data))

def upscale_image_with_imagen(
    image_path: str,
    project_id: str,
    region: str,
    upscale_factor: str = "x4",
    output_mime_type: str = "image/png",
    compression_quality: int = 90,
    prompt: str = "Upscale the image",
    credentials = None
) -> Image.Image:
    """
    Upscale an image using Google Imagen 4.0 API.
    
    Args:
        image_path: Path to input image
        project_id: Google Cloud project ID
        region: Vertex AI region
        upscale_factor: "x2", "x3", or "x4"
        output_mime_type: "image/png" or "image/jpeg"
        compression_quality: JPEG quality (0-100)
        prompt: Text prompt for upscaling
        credentials: Google auth credentials
    
    Returns:
        PIL Image of the upscaled result
    """
    # Build API endpoint
    endpoint = f"https://{region}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{region}/publishers/google/models/imagen-4.0-upscale-preview:predict"
    
    # Get fresh access token
    access_token = get_access_token(credentials)
    
    # Encode image
    print(f"  Encoding image...")
    image_base64 = encode_image_to_base64(image_path)
    
    # Build request body (PNG does not accept compressionQuality)
    output_options: dict = {"mimeType": output_mime_type}
    if output_mime_type == "image/jpeg":
        output_options["compressionQuality"] = compression_quality

    request_body = {
        "instances": [
            {
                "prompt": prompt,
                "image": {
                    "bytesBase64Encoded": image_base64
                }
            }
        ],
        "parameters": {
            "mode": "upscale",
            "upscaleConfig": {
                "upscaleFactor": upscale_factor
            },
            "outputOptions": output_options
        }
    }
    
    # Make API request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json; charset=utf-8"
    }
    
    print(f"  Calling Imagen API ({upscale_factor} upscale)...")
    start_time = time.time()
    
    response = requests.post(
        endpoint,
        headers=headers,
        json=request_body,
        timeout=300  # 5 minute timeout
    )
    
    api_time = time.time() - start_time
    print(f"  API response received in {api_time:.1f}s")
    
    # Check for errors
    if response.status_code != 200:
        error_msg = response.text
        try:
            error_json = response.json()
            if "error" in error_json:
                error_msg = error_json["error"].get("message", error_msg)
        except:
            pass
        raise Exception(f"API error ({response.status_code}): {error_msg}")
    
    # Parse response
    result = response.json()
    
    if "predictions" not in result or len(result["predictions"]) == 0:
        raise Exception("No predictions in API response")
    
    # Decode the upscaled image
    prediction = result["predictions"][0]
    upscaled_base64 = prediction.get("bytesBase64Encoded")
    
    if not upscaled_base64:
        raise Exception("No image data in prediction")
    
    return decode_base64_to_image(upscaled_base64)


def upscale_and_resize(
    image_path: str,
    target_width: int,
    target_height: int,
    output_name: str,
    project_id: str,
    region: str,
    upscale_factor: str = "x4",
    output_mime_type: str = "image/png",
    compression_quality: int = 90,
    prompt: str = "Upscale the image",
    credentials = None
) -> str:
    """
    Upscale image with Imagen API and resize to exact target dimensions.
    
    Args:
        image_path: Path to input image
        target_width: Target width in pixels
        target_height: Target height in pixels
        output_name: Output filename (without extension)
        project_id: Google Cloud project ID
        region: Vertex AI region
        upscale_factor: "x2", "x3", or "x4"
        output_mime_type: "image/png" or "image/jpeg"
        compression_quality: JPEG quality (0-100)
        prompt: Text prompt for upscaling
        credentials: Google auth credentials
    
    Returns:
        Path to output image
    """
    start_time = time.time()
    
    # Resolve image path
    image_path = Path(image_path)
    if not image_path.is_absolute():
        image_path = REPO_ROOT / image_path
    
    if not image_path.exists():
        raise FileNotFoundError(f"Input file not found: {image_path}")
    
    print(f"Processing: {image_path.name}")
    print(f"Target output: {target_width}x{target_height}px")
    
    # Get input dimensions
    with Image.open(image_path) as img:
        w, h = img.size
        print(f"Input size: {w}x{h}")
    
    # Calculate upscaled size
    scale = int(upscale_factor[1])  # Extract number from "x4" -> 4
    upscaled_w, upscaled_h = w * scale, h * scale
    print(f"Expected upscaled size: {upscaled_w}x{upscaled_h}")
    
    # Call Imagen API
    upscaled_img = upscale_image_with_imagen(
        image_path=str(image_path),
        project_id=project_id,
        region=region,
        upscale_factor=upscale_factor,
        output_mime_type=output_mime_type,
        compression_quality=compression_quality,
        prompt=prompt,
        credentials=credentials
    )
    
    actual_w, actual_h = upscaled_img.size
    print(f"Actual upscaled size: {actual_w}x{actual_h}")
    
    # Resize to exact target dimensions
    print(f"Resizing to target: {target_width}x{target_height}...")
    final_img = upscaled_img.resize((target_width, target_height), Image.LANCZOS)
    
    # Save output
    output_path = OUTPUT_DIR / f"{output_name}.png"
    final_img.save(str(output_path), "PNG")
    
    total_time = time.time() - start_time
    
    print(f"\n\u2705 Done!")
    print(f"Output: {output_path}")
    print(f"Final size: {target_width}x{target_height}")
    print(f"Total time: {total_time:.1f}s")
    
    return str(output_path)

print("\u2705 Functions defined!")

✅ Functions defined!


In [5]:
# @title Run Batch Upscaling

print("=" * 60)
print("BATCH UPSCALING WITH IMAGEN 4.0")
print("=" * 60)

results = []

for i, config in enumerate(IMAGE_CONFIGS, 1):
    print(f"\n{'=' * 60}")
    print(f"[{i}/{len(IMAGE_CONFIGS)}] {config['image_id']}")
    print(f"Target: {config['width_inches']}x{config['height_inches']}\" at {DPI} DPI")
    print(f"{'=' * 60}")
    
    try:
        output_name = f"{config['image_id']}_imagen_4x_eval"
        
        output_path = upscale_and_resize(
            image_path=config["input_path"],
            target_width=config["target_width"],
            target_height=config["target_height"],
            output_name=output_name,
            project_id=PROJECT_ID,
            region=REGION,
            upscale_factor=UPSCALE_FACTOR,
            output_mime_type=OUTPUT_MIME_TYPE,
            compression_quality=COMPRESSION_QUALITY,
            prompt=UPSCALE_PROMPT,
            credentials=credentials
        )
        
        results.append({
            "image_id": config["image_id"],
            "status": "Success",
            "output": output_path,
            "size": f"{config['width_inches']}x{config['height_inches']}\""
        })
        
    except Exception as e:
        print(f"\n\u274c Error: {e}")
        results.append({
            "image_id": config["image_id"],
            "status": "Failed",
            "error": str(e)
        })

# Summary
print(f"\n\n{'=' * 60}")
print("BATCH COMPLETE - SUMMARY")
print(f"{'=' * 60}")

success_count = sum(1 for r in results if r["status"] == "Success")
print(f"\nProcessed: {success_count}/{len(results)} images successfully\n")

for r in results:
    if r["status"] == "Success":
        print(f"\u2705 {r['image_id'][:20]}... -> {r['size']} -> {Path(r['output']).name}")
    else:
        print(f"\u274c {r['image_id'][:20]}... -> {r.get('error', 'Unknown error')}")

BATCH UPSCALING WITH IMAGEN 4.0

[1/3] 7a49a140-beee-4141-9593-3fddb9194240
Target: 10x20" at 150 DPI
Processing: 7a49a140-beee-4141-9593-3fddb9194240.jpg
Target output: 1500x3000px
Input size: 682x1024
Expected upscaled size: 2728x4096
  Encoding image...
  Calling Imagen API (x4 upscale)...
  API response received in 16.3s
Actual upscaled size: 2728x4096
Resizing to target: 1500x3000...

✅ Done!
Output: C:\Users\Armaan\Desktop\Artinafti\4xoutput\7a49a140-beee-4141-9593-3fddb9194240_imagen_4x_eval.png
Final size: 1500x3000
Total time: 17.0s

[2/3] b46bc519-9b4b-487f-b8d4-b95195d7e02e
Target: 17x10" at 150 DPI
Processing: b46bc519-9b4b-487f-b8d4-b95195d7e02e.jpg
Target output: 2550x1500px
Input size: 1024x587
Expected upscaled size: 4096x2348
  Encoding image...
  Calling Imagen API (x4 upscale)...
  API response received in 13.8s
Actual upscaled size: 4096x2348
Resizing to target: 2550x1500...

✅ Done!
Output: C:\Users\Armaan\Desktop\Artinafti\4xoutput\b46bc519-9b4b-487f-b8d4-b95195d7

## Single Image Processing (Optional)

Use the cell below if you want to process a single image with custom settings.

In [6]:
# @title Single Image Upscale (Optional)

# Uncomment and modify to process a single image:

# single_result = upscale_and_resize(
#     image_path="input/your_image.jpg",
#     target_width=1650,   # 11 inches * 150 DPI
#     target_height=2100,  # 14 inches * 150 DPI
#     output_name="your_image_imagen",
#     project_id=PROJECT_ID,
#     region=REGION,
#     upscale_factor="x4",
#     output_mime_type="image/png",
#     compression_quality=90,
#     prompt="Upscale with sharp details",
#     credentials=credentials
# )

## Simple Upscale (No Resize)

Use this if you just want to upscale without resizing to a specific target size.

In [7]:
# @title Simple Upscale (No Target Size)

def simple_upscale(
    image_path: str,
    output_name: str,
    upscale_factor: str = "x4"
) -> str:
    """
    Simple upscale without resizing.
    
    Args:
        image_path: Path to input image
        output_name: Output filename (without extension)
        upscale_factor: "x2", "x3", or "x4"
    
    Returns:
        Path to output image
    """
    image_path = Path(image_path)
    if not image_path.is_absolute():
        image_path = REPO_ROOT / image_path
    
    print(f"Processing: {image_path.name}")
    
    # Get input dimensions
    with Image.open(image_path) as img:
        w, h = img.size
        print(f"Input size: {w}x{h}")
    
    # Call Imagen API
    upscaled_img = upscale_image_with_imagen(
        image_path=str(image_path),
        project_id=PROJECT_ID,
        region=REGION,
        upscale_factor=upscale_factor,
        output_mime_type=OUTPUT_MIME_TYPE,
        compression_quality=COMPRESSION_QUALITY,
        prompt=UPSCALE_PROMPT,
        credentials=credentials
    )
    
    actual_w, actual_h = upscaled_img.size
    print(f"Output size: {actual_w}x{actual_h}")
    
    # Save output
    output_path = OUTPUT_DIR / f"{output_name}.png"
    upscaled_img.save(str(output_path), "PNG")
    
    print(f"\n\u2705 Saved to: {output_path}")
    return str(output_path)

# Uncomment to use:
# result = simple_upscale(
#     image_path="input/your_image.jpg",
#     output_name="your_image_upscaled",
#     upscale_factor="x4"
# )